In [152]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import QuantLib as ql
import pricing_kit as pk
import matplotlib.pyplot as plt
import datetime

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [153]:
inputData = pd.read_excel('InputData.xlsx', index_col=None)
inputData

,Deal,Product type,Valuation Date,Start Date,Maturity (Y),Nominal,Fixed Rate,Fixed Leg Tenor,Index,Swap Type
0,Vanilla swap,Rates,2020-07-13,2020-07-08,7,600000,0.04,1Y,Euribor6M,Receiver
1,Vanilla swap,Rates,2020-07-13,2020-07-08,5,1000000,0.04,6M,Euribor3M,Payer


### Schedule or Grid Method

In [154]:
#Standardised initial margin schedule
im_Schedule = pd.DataFrame(np.array([["[0,2]",0.02,0.15,0.06,0.01,0.15],["[2,5]",0.05,0.15,0.06,0.02,0.15],[">5",0.1,0.15,0.06,0.04,0.15]]),
              columns = ["Maturity (in years)", "Credit","Equity","FX","Rates", "Other"])
im_Schedule = im_Schedule.astype({"Credit": float, "Equity": float, "FX": float, "Rates": float, "Other": float})
im_Schedule

,Maturity (in years),Credit,Equity,FX,Rates,Other
0,"[0,2]",0.02,0.15,0.06,0.01,0.15
1,"[2,5]",0.05,0.15,0.06,0.02,0.15
2,>5,0.10,0.15,0.06,0.04,0.15


In [148]:
#Evaluation date
valuationDate = ql.Date(inputData.loc[0, "Valuation Date"].strftime("%Y-%m-%d"), '%Y-%m-%d')
StartDateVect = [ql.Date(inputData.loc[i, "Start Date"].strftime("%Y-%m-%d"), '%Y-%m-%d') for i in range(inputData.shape[0])]

#MarketData set up
rateHandleVect = [ql.QuoteHandle(ql.SimpleQuote(inputData.loc[i, "Fixed Rate"])) for i in range(inputData.shape[0])]
dayCounter = ql.Actual365Fixed()
YieldTermStructVect =  [ql.FlatForward(StartDateVect[i], rateHandleVect[i], dayCounter) for i in range(inputData.shape[0])]
#YieldTermStructVect =  [ql.FlatForward(valuationDate, rateHandleVect[i], dayCounter) for i in range(inputData.shape[0])]

for i in range(inputData.shape[0]): 
    YieldTermStructVect[i].enableExtrapolation()
    
YieldTermStructHandleVect = [ql.RelinkableYieldTermStructureHandle(YieldTermStructVect[i]) for i in range(inputData.shape[0])]
euribor6m = [ql.Euribor6M(YieldTermStructHandleVect[i]) for i in range(inputData.shape[0])]

#set a dummy portfolio from input data
portfolio = [pk.swapGenerator(ql.Date(inputData.loc[i, "Start Date"].strftime("%Y-%m-%d"),'%Y-%m-%d') + ql.Period("2d"),
                          ql.Period(str(inputData.loc[i, "Maturity (Y)"])+"Y"), np.float64(inputData.loc[i, "Nominal"]), 
                           ql.SimpleQuote(inputData.loc[i, "Fixed Rate"]).value(), euribor6m[i], 
                          ql.VanillaSwap.Receiver if inputData.loc[i, "Swap Type"]=="Receiver" else ql.VanillaSwap.Payer) 
             for i in range(inputData.shape[0])]

In [17]:
import pandas as pd
def get_spot_rates(yieldcurve):
    keytenors = [0, 6, 12, 18, 24]
    spotDate = yieldcurve.referenceDate()
    dates = [ql.TARGET().advance(spotDate, t, ql.Months) for t in keytenors ]
    rates = [yieldcurve.zeroRate(t, ql.Actual365Fixed(), ql.Continuous) for t in dates ]
    eq_rate = [rates[d].equivalentRate(ql.Actual365Fixed(), ql.Continuous, ql.Semiannual, spotDate , dates[d]).rate() for d in range(len(dates))]
    spots = [eq_r for eq_r in eq_rate]
    df = pd.DataFrame(list(zip(dates, spots)),columns=["The Dates","The Rates"], index=['']*len(dates) )
    return df

In [158]:
def IM_Grid_method(im_Schedule: pd.DataFrame, portfolio, inputData: pd.DataFrame, valuationDate): 
    """
    Return the initial margin of a portfolio following the schedule method
        parameters: 
            standardGrid(DataFrame): Standard margin ratio grid per product class
            inputData(DataFrame): portfolio database with attributes:Notiional, Maturity, Product type...
            portfolio: list of quantlib financial product objects
            
    """
    init_m = 0
    for j in ["Credit", "Equity", "FX", "Rates", "other"]: 
        portfolio_filtered_indexes = inputData[inputData["Product type"]==j].index.values.tolist()
        for i in portfolio_filtered_indexes: 
            if inputData.loc[i, "Maturity (Y)"]<=5 and inputData.loc[i, "Maturity (Y)"] >2:
                margin_ratio = im_Schedule.loc[1,j]
                init_m += margin_ratio * inputData.loc[i,"Nominal"]
            elif inputData.loc[i, "Maturity (Y)"]<=0 and inputData.loc[i, "Maturity (Y)"] >2:
                margin_ratio = im_Schedule.loc[0,j]
                init_m += margin_ratio * inputData.loc[i,"Nominal"]
            else: 
                margin_ratio = im_Schedule.loc[2,j]
                init_m += margin_ratio * inputData.loc[i,"Nominal"]
                
    ql.Settings.instance().evaluationDate = valuationDate
    
    portfolio_npv=np.zeros((len(portfolio)))
    
    #Market data setup
    rateHandleVect = [ql.QuoteHandle(ql.SimpleQuote(inputData.loc[i, "Fixed Rate"])) for i in range(inputData.shape[0])]
    dayCounter = ql.Actual365Fixed()
    YieldTermStructVect =  [ql.FlatForward(valuationDate, rateHandleVect[i], dayCounter) for i in range(inputData.shape[0])]
    YieldTermStructHandleVect = [ql.RelinkableYieldTermStructureHandle(YieldTermStructVect[i]) for i in range(inputData.shape[0])]
    
    for i in range(len(portfolio_npv)): 
        datesList=[ql.Date(j,7,2020) for j in range(8,13) if euribor6m[i].isValidFixingDate(ql.Date(j,7,2020))]    
        euribor6m[i].addFixings(datesList, [0.05]*len(datesList))
        
        YieldTermStructVect[i].enableExtrapolation()
        YieldTermStructHandleVect[i].linkTo(YieldTermStructVect[i])
        engine = ql.DiscountingSwapEngine(YieldTermStructHandleVect[i])
        portfolio[i][0].setPricingEngine(engine)
        portfolio_npv[i] = portfolio[i][0].NPV()
        
    NGR = np.absolute(np.sum(portfolio_npv))/np.sum(np.absolute(portfolio_npv)) if np.sum(portfolio_npv)/np.sum(np.absolute(portfolio_npv))>0 else 0
    
    return [0.4*init_m + 0.6*init_m*NGR, init_m, NGR, portfolio_npv[0], portfolio_npv[1]]

In [162]:
portfolio_filtered_indexes = inputData[inputData["Product type"]=="Rates"].index.values.tolist()
portfolio_filtered_indexes

[0, 1]

In [157]:
portfolio

[(<QuantLib.QuantLib.VanillaSwap; proxy of <Swig Object of type 'boost::shared_ptr< VanillaSwap > *' at 0x000002395C3FF750> >,
  [Date(8,7,2020),
   Date(9,7,2020),
   Date(8,1,2021),
   Date(8,7,2021),
   Date(10,1,2022),
   Date(8,7,2022),
   Date(10,1,2023),
   Date(10,7,2023),
   Date(10,1,2024),
   Date(10,7,2024),
   Date(9,1,2025),
   Date(10,7,2025),
   Date(8,1,2026),
   Date(9,7,2026),
   Date(8,1,2027)]),
 (<QuantLib.QuantLib.VanillaSwap; proxy of <Swig Object of type 'boost::shared_ptr< VanillaSwap > *' at 0x000002395CB7EC30> >,
  [Date(8,7,2020),
   Date(7,1,2021),
   Date(8,7,2021),
   Date(6,1,2022),
   Date(7,7,2022),
   Date(6,1,2023),
   Date(6,7,2023),
   Date(8,1,2024),
   Date(8,7,2024),
   Date(8,1,2025)])]

In [141]:
valuationDate =  ql.Date(12,9,2020)
ql.Settings.instance().evaluationDate = valuationDate
    
portfolio_npv=np.zeros((len(portfolio)))
    
rateHandleVect = [ql.QuoteHandle(ql.SimpleQuote(inputData.loc[i, "Fixed Rate"])) for i in range(inputData.shape[0])]
dayCounter = ql.Actual365Fixed()
YieldTermStructVect =  [ql.FlatForward(valuationDate, rateHandleVect[i], dayCounter) for i in range(inputData.shape[0])]
YieldTermStructHandleVect = [ql.RelinkableYieldTermStructureHandle(YieldTermStructVect[i]) for i in range(inputData.shape[0])]
    
for i in range(len(portfolio_npv)): 
    datesList=[ql.Date(j,7,2020) for j in range(8,13) if euribor6m[i].isValidFixingDate(ql.Date(j,7,2020))]    
    euribor6m[i].addFixings(datesList, [0.05]*len(datesList))
        
    YieldTermStructVect[i].enableExtrapolation()
    YieldTermStructHandleVect[i].linkTo(YieldTermStructVect[i])
    engine = ql.DiscountingSwapEngine(YieldTermStructHandleVect[i])
    portfolio[i][0].setPricingEngine(engine)
    portfolio_npv[i] = portfolio[i][0].NPV()
    
A = np.abs(np.sum(portfolio_npv))
B = np.sum((np.absolute(portfolio_npv)))
    
print(portfolio_npv)
print (A/B)

[-6042.33429644  8869.28253389]
0.1895802627990195


In [159]:
IM_Grid_method(im_Schedule, portfolio, inputData, ql.Date(20,8,2020))

[22604.918937894185,
 44000.0,
 0.18958026279902218,
 -6027.123480988608,
 8846.955232358247]